## Scraping Berlingske for Novo Nordisk Articles

In [125]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import time
import re
import requests
import pandas as pd
import nltk

In [44]:
# Set up the driver and navigate to the URL
url = 'https://www.berlingske.dk'
driver = webdriver.Chrome()  # Ensure you have the ChromeDriver set up correctly
driver.get(url)

# Find and click on the "Allow Cookies" button
allow_cookies_button = driver.find_element(By.CSS_SELECTOR, "#CybotCookiebotDialogBodyButtonDecline")
allow_cookies_button.click()

#Find and click the menu button
menu_data_id = "menuToggler"
menu_element = driver.find_element(By.CSS_SELECTOR, f"[data-id='{menu_data_id}']")
menu_element.click()

# Locate the search input element by its ID
search_input_id = "header-search"
search_input = driver.find_element(By.ID, search_input_id)

# Perform a search action in the search input
search_input.send_keys("Novo Nordisk")
search_input.submit()

In [45]:
# Loop to click the "Load More" button and stop clicking when it is no longer possible
for _ in range(1000): 
    try:
        load_button = driver.find_element(By.XPATH, '/html/body/div[2]/div[4]/div[2]/div/div/div/div[2]/button')
        load_button.click()
        time.sleep(0.2)
    except (NoSuchElementException, ElementNotInteractableException):
        print("Load More button not found or not clickable. Exiting loop.")
        break

In [99]:
#Because we use the Selenium package and don't make use of the get response, we cannot include a header with contact information. Due to the structure of the website, we found that the most effecient way to obtain data was via Selenium.
# Get the page source
page_source = driver.page_source

# Use Beautiful Soup to parse the HTML
soup = BeautifulSoup(page_source, "html.parser")


# Find all article containers with the specified class
article_containers = soup.find_all('div', class_='teaser teaser--border-bottom teaser--search d-flex flex-row w-100')

# Lists to hold extracted data
titles = []
dates = []

# Loop through the article containers and extract title and date
for container in article_containers:
    title = container.find('a', class_='teaser__title-link').text.strip()
    date = container.find('div', class_='teaser__date').text.strip()
    
    titles.append(title)
    dates.append(date)

# Convert the lists to a pandas DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Date': dates})

df


,Title,Date
0,Han er topchef for 55.000 mennesker og introve...,11. august 2023
1,Business-overblik: Lauritz-manøvre skjulte dun...,11. august 2023
2,Novo vokser og vokser. Er nu mere værd end Por...,10. august 2023
3,Lars Fruergaard affejer kritik af mangel på di...,10. august 2023
4,Business-update: »En narkobande af Mærsk-ansat...,10. august 2023
...,...,...
9456,Kendt dansk kommunikationsbureau går i opløsning,1. oktober 2015
9457,"»Jeg kan godt knibe mig i armen og spørge, hvo...",1. oktober 2015
9458,Minister: Ny prisportal kan skærpe konkurrence...,1. oktober 2015
9459,Fra mælkemand til kødkonge: Her er Danish Crow...,1. oktober 2015


In [100]:
#To filter the articles, we make a list of keywords
#Initially, this list must inclide the names of all Novo Nordisk medical produkts. These can be obtained as follows: 


# URL to scrape from
url = "https://min.medicin.dk/Adresser/Medicinvirksomheder/126/"

# Send a GET request
response = requests.get(url, headers={'name': 'Lea Lund Jeppesen', 'e-mail':'snk956@alumni.ku.dk', 'purpose':'University of Copenhagen Social Data Science exam project'})

# Parse the content with BeautifulSoup
soup = BeautifulSoup(response.content, 'lxml')

# Find all product elements
product_elements = soup.find_all('a', class_='glob-search_link')

# Extract the product names from the title attribute
product_names = [product['title'] for product in product_elements]

# Removing special characters and numbers from each product name by using Regular Expressions
product_names_cleaned = [re.sub(r'[^a-zA-Z\s]', '', name) for name in product_names_updated]

# For the 28th element, split it into 2 by space
words = product_names_cleaned[28].split()

# Replace the 28th name with the first word
product_names_cleaned[28] = words[0]

# Insert the second word as the 29th element
product_names_cleaned.insert(30, words[1])

#Print the list
product_names_cleaned


['Activelle',
 'Actrapid',
 'Esperoct',
 'Estrofem',
 'Fiasp',
 'GlucaGen',
 'Insulatard',
 'Kliogest',
 'Levemir',
 'Mixtard ',
 'Norditropin',
 'NovoEight',
 'Novofem',
 'NovoMix',
 'NovoNorm',
 'NovoRapid',
 'Novoseven',
 'NovoThirtee',
 'Ozempic',
 'Refixia',
 'Rybelsus',
 'Ryzodeg',
 'Saxenda',
 'Sogroya',
 'Tresiba',
 'Trisekvens',
 'Vagifem',
 'Victoza',
 'Wegovy',
 'Xultophy',
 'Flextouch']

In [101]:
# Define additional keywords, these are determined manually by research on Novo Nordisk medical expertise
keywords = ["Novo", "Nordisk", "insulin", "fedme", "diabetes", "vægttab", "præparat", "overvægt", "slank", "medicin", "pille"]

# We then append the product_names_cleaned list to the keywords list
for product in product_names_cleaned:
    keywords.append(product)

print(keywords)

['Novo', 'Nordisk', 'insulin', 'fedme', 'diabetes', 'vægttab', 'præparat', 'overvægt', 'slank', 'medicin', 'pille', 'Activelle', 'Actrapid', 'Esperoct', 'Estrofem', 'Fiasp', 'GlucaGen', 'Insulatard', 'Kliogest', 'Levemir', 'Mixtard ', 'Norditropin', 'NovoEight', 'Novofem', 'NovoMix', 'NovoNorm', 'NovoRapid', 'Novoseven', 'NovoThirtee', 'Ozempic', 'Refixia', 'Rybelsus', 'Ryzodeg', 'Saxenda', 'Sogroya', 'Tresiba', 'Trisekvens', 'Vagifem', 'Victoza', 'Wegovy', 'Xultophy', 'Flextouch']


In [102]:
# Convert the keywords to lowercase for case-insensitive matching
keywords_lower = [word.lower() for word in keywords]

# Filter the dataframe based on the condition
df_filtered = df[df['Title'].str.lower().str.contains('|'.join(keywords_lower))]

#Reset index
df_filtered = df_filtered.reset_index(drop=True)

#Transform the Date column to the same format as Yahoo Finance data
# Step 1: Translate the month names to English
def translate_month_to_english(date_string):
    months_danish = ["januar", "februar", "marts", "april", "maj", "juni", "juli", 
                     "august", "september", "oktober", "november", "december"]
    months_english = ["January", "February", "March", "April", "May", "June", "July", 
                      "August", "September", "October", "November", "December"]
    for danish, english in zip(months_danish, months_english):
        date_string = date_string.replace(danish, english)
    return date_string

# Apply the translation function to the 'Date' column
df_filtered['Date'] = df_filtered['Date'].apply(translate_month_to_english)

# Step 2 & 3: Convert the translated date string into a datetime object and then format it
df_filtered['Date'] = pd.to_datetime(df_filtered['Date'], format='%d. %B %Y').dt.strftime('%Y-%m-%d')


In [103]:
#Now we want to process the text data

#We define a processing function that carries out the wanted steps 
def process_text(cell_text): 
    cell_text=cell_text.lower() #Change to lower case
    cell_text=re.sub(r'[^\w\s]','',cell_text) #Remove all characters that are not words or numbers
    cell_text=cell_text.split() #Split at whitespace with the split() method (i.e., tokenization)
    return cell_text

#We now apply the process function to the titles 
df_filtered['Title'] = df_filtered['Title'].apply(process_text)
df_filtered

,Title,Date
0,"[novo, vokser, og, vokser, er, nu, mere, værd,...",2023-08-10
1,"[lars, fruergaard, affejer, kritik, af, mangel...",2023-08-10
2,"[novo, nordisk, kæmper, med, at, følge, den, s...",2023-08-10
3,"[salg, af, novoslankemiddel, boomer, mere, end...",2023-08-10
4,"[novo, nordisk, leverer, atter, milliardoversk...",2023-08-10
...,...,...
1713,"[novo, nordisk, aktien, ramt, af, nedtur]",2015-10-14
1714,"[novoboss, kåret, som, verdens, bedste, topchef]",2015-10-13
1715,"[novoboss, kåret, som, verdens, bedste, topchef]",2015-10-13
1716,"[novozymestopchef, ja, vi, har, fået, en, rids...",2015-10-11


In [119]:
#We want to remove stopwords
nltk.download('stopwords')
stop = stopwords.words('danish')

#Remove stopwards
df_filtered['title_nostop'] = df_filtered['Title'].apply(lambda title: [i for i in title if i not in stop])

#We ensure that some stopwords were actually removed
print(len(df_filtered['Title'].iloc[0]), len(df_filtered['title_nostop'].iloc[0]))

#Print the data_frame
df_filtered


15 10


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leajeppesen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Title,Date,title_nostop,title_stemmed
0,"[novo, vokser, og, vokser, er, nu, mere, værd,...",2023-08-10,"[novo, vokser, vokser, mere, værd, porsche, vo...","[novo, voks, voks, mer, værd, porsch, volkswag..."
1,"[lars, fruergaard, affejer, kritik, af, mangel...",2023-08-10,"[lars, fruergaard, affejer, kritik, mangel, di...","[lar, fruergaard, affej, kritik, mangel, diabe..."
2,"[novo, nordisk, kæmper, med, at, følge, den, s...",2023-08-10,"[novo, nordisk, kæmper, følge, store, efterspø...","[novo, nordisk, kæmp, følg, stor, efterspørgsel]"
3,"[salg, af, novoslankemiddel, boomer, mere, end...",2023-08-10,"[salg, novoslankemiddel, boomer, mere, seksdob...","[salg, novoslankemiddel, boom, mer, seksdobl]"
4,"[novo, nordisk, leverer, atter, milliardoversk...",2023-08-10,"[novo, nordisk, leverer, atter, milliardoversk...","[novo, nordisk, lev, att, milliardoverskud, øg..."
...,...,...,...,...
1713,"[novo, nordisk, aktien, ramt, af, nedtur]",2015-10-14,"[novo, nordisk, aktien, ramt, nedtur]","[novo, nordisk, akti, ramt, nedtur]"
1714,"[novoboss, kåret, som, verdens, bedste, topchef]",2015-10-13,"[novoboss, kåret, verdens, bedste, topchef]","[novobos, kår, verd, bedst, topchef]"
1715,"[novoboss, kåret, som, verdens, bedste, topchef]",2015-10-13,"[novoboss, kåret, verdens, bedste, topchef]","[novobos, kår, verd, bedst, topchef]"
1716,"[novozymestopchef, ja, vi, har, fået, en, rids...",2015-10-11,"[novozymestopchef, ja, fået, ridse, tilliden]","[novozymestopchef, ja, fået, rids, tillid]"


In [120]:
nltk.download('omw-1.4')
nltk.download('wordnet')

# Initialize the Danish SnowballStemmer
stemmer = SnowballStemmer("danish")

# Define the stemming function
def stem_text(tokens):
    return [stemmer.stem(token) for token in tokens]

# Apply the function to each row of the 'title_nostop' column
df_filtered['title_stemmed'] = df_filtered['title_nostop'].apply(stem_text)

df_filtered

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/leajeppesen/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leajeppesen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Title,Date,title_nostop,title_stemmed
0,"[novo, vokser, og, vokser, er, nu, mere, værd,...",2023-08-10,"[novo, vokser, vokser, mere, værd, porsche, vo...","[novo, voks, voks, mer, værd, porsch, volkswag..."
1,"[lars, fruergaard, affejer, kritik, af, mangel...",2023-08-10,"[lars, fruergaard, affejer, kritik, mangel, di...","[lar, fruergaard, affej, kritik, mangel, diabe..."
2,"[novo, nordisk, kæmper, med, at, følge, den, s...",2023-08-10,"[novo, nordisk, kæmper, følge, store, efterspø...","[novo, nordisk, kæmp, følg, stor, efterspørgsel]"
3,"[salg, af, novoslankemiddel, boomer, mere, end...",2023-08-10,"[salg, novoslankemiddel, boomer, mere, seksdob...","[salg, novoslankemiddel, boom, mer, seksdobl]"
4,"[novo, nordisk, leverer, atter, milliardoversk...",2023-08-10,"[novo, nordisk, leverer, atter, milliardoversk...","[novo, nordisk, lev, att, milliardoverskud, øg..."
...,...,...,...,...
1713,"[novo, nordisk, aktien, ramt, af, nedtur]",2015-10-14,"[novo, nordisk, aktien, ramt, nedtur]","[novo, nordisk, akti, ramt, nedtur]"
1714,"[novoboss, kåret, som, verdens, bedste, topchef]",2015-10-13,"[novoboss, kåret, verdens, bedste, topchef]","[novobos, kår, verd, bedst, topchef]"
1715,"[novoboss, kåret, som, verdens, bedste, topchef]",2015-10-13,"[novoboss, kåret, verdens, bedste, topchef]","[novobos, kår, verd, bedst, topchef]"
1716,"[novozymestopchef, ja, vi, har, fået, en, rids...",2015-10-11,"[novozymestopchef, ja, fået, ridse, tilliden]","[novozymestopchef, ja, fået, rids, tillid]"
